### Extract Structured Data From Text using Kor & LangChain


In [ ]:
!pip install --user -U nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
import nltk
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters

In [ ]:
!pip -q install langchain openai google-search-results tiktoken
!pip -q install kor markdownify

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
# Kor!
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

# LangChain Models
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# Standard Helpers
import pandas as pd
import requests
import time
import json
from datetime import datetime

# Text Helpers
from bs4 import BeautifulSoup
from markdownify import markdownify as md

# For token counting
from langchain.callbacks import get_openai_callback

def printOutput(output):
 print(json.dumps(output,sort_keys=True, indent=3))

In [ ]:
openai_api_key = 'enter your key here'

In [ ]:
!pip show openai

Name: openai
Version: 0.28.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, requests, tqdm
Required-by: 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using GPT-4 model

In [ ]:
llm = ChatOpenAI(
#  model_name="gpt-3.5-turbo", # Cheaper but less reliable or "gpt-4", or "gpt-4-1106-preview"
 model_name="gpt-3.5-turbo",
 temperature=0,
 max_tokens=2000,
 openai_api_key=openai_api_key
)

<ipython-input-8-4132f40993ae>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(


### Building Schema for KOR

In [ ]:
schema = Object(
 id="Triple",
 description="Your task is to recognize the triple (entity 1 - predicate - entity 2) connection in each sentence if they exist.",
 attributes=[
  Text(
   id="Predicate",
   description= f"""Your task is to identify the main action in triples (entity 1 - action - entity 2) in each individual sentence following these steps:
1 - Segment the text into individual sentences.

2 - Infer the predicate in each sentence that has been splitted. Then extract only the verb phrase in the predicate. Predicate is the geographical connection between two geographical entities.
Perform step 2 using the reasoning in this example:

Example: Middlesex County is located in New Jersey.
Reasoning: 'is located in' is a predicate and belongs to the triple: Middlesex County (GPE) - is located in - New Jersey (GPE).
Output: is located in

If no action that satisfies the condition in step 2 is detected, or no entities detected, output 'None'.
"""
 ),
  Text(
   id="Entity 1",
   description=f"""Your task is to identify the first named entity on each individual sentence following these steps:
1 - Segment the text into individual sentences.

2 - Identify the first named entity that share the entity 1 - action - entity 2 relationship in each sentence that has been splitted.
Ignore named entities that do not belong in a triple.

3 - Perform NER on entity 1 with one of these 2 labels only: GPE for nations, states, cities, LOC for non-GPE location.

Perform step 2 and 3 using the reasoning in this example:

Example: 'Champaign County is part of Illinois.'
Reasoning: 'Champaign county' appears first in a triple: Champaign County - is part of - Illinois.
It is a county name, which corresponds with “LOC” in the given entity types.
Output: Champaign County (LOC)

If no named entity 1 in the triple is detected, output 'None'.

"""
 ),
  Text(
   id="Entity 2",
   description=f"""Your task is to identify the second named entity on each individual sentence following these steps:
1 - Segment the text into individual sentences.

2 - Identify the second named entity that share the entity 1 - predicate - entity 2 (triple) in each sentence that has been splitted.
Ignore named entities that do not belong in a triple.

3 - Perform NER on entity 2 with one of these 2 labels only: GPE for nations, states, cities, LOC for non-GPE location.

Perform step 2 and 3 using the reasoning in this example:

Example: 'Champaign County is part of Illinois.'
Reasoning: 'Illinois' appears second in a triple: Champaign County - is part of - Illinois.
It is a state's name, which corresponds with GPE in the given entity types.
Output: Illinois (GPE)

If no named entity 2 in the triple is detected, output 'None'.
"""
  ),
  Text(
   id="Output",
   description=f"""
Final step - Output triples of entity 1 - predicate - entity 2 for each sentence in a structured JSON format with 'Predicate', 'Entity 1', 'Entity 2' as the only 3 keys.
When multiple entities are listed, generate separate entity pairs.
For sentences without actions or any of the entities, output as instructed.
Example of separate entity pairs:

Example output:

Sentence - Platte County is part of Wyoming.
Predicate: is part of
Entity 1: Platte County (GPE)
Entity 2: Wyoming (GPE)

Sentence - Shannon County is part of Missouri.
Predicate: is part of
Entity 1: Shannon County (GPE)
Entity 2: Missouri (GPE)

Sentence - Vermilion County is part of Illinois.
Predicate: is part of
Entity 1: Vermilion County (GPE)
Entity 2: Illinois (GPE)

Sentence - Bent County is part of Colorado.
Predicate: is part of
Entity 1: Bent County (GPE)
Entity 2: Colorado (GPE)

Sentence - Hillsborough County is part of Florida.
Predicate: is part of
Entity 1: Hillsborough County (GPE)
Entity 2: Florida (GPE)

Sentence - Kingfisher County is part of Oklahoma.
Predicate: is part of
Entity 1: Kingfisher County (GPE)
Entity 2: Oklahoma (GPE)

Sentence - Doddridge County is part of West Virginia.
Predicate: is part of
Entity 1: Doddridge County (GPE)
Entity 2: West Virginia (GPE)

Sentence - Charlton County is part of Georgia.
Predicate: is part of
Entity 1: Charlton County (GPE)
Entity 2: Georgia (GPE)

Sentence - Boone County is part of Kentucky.
Predicate: is part of
Entity 1: Boone County (GPE)
Entity 2: Kentucky (GPE)

Sentence - Bergen County is part of New Jersey.
Predicate: is part of
Entity 1: Bergen County (GPE)
Entity 2: New Jersey (GPE)

Note: Ensure that each output contains only one instance of each attribute.
"""
  ),
 ],
 many=True,
)

In [ ]:
# New data as training
# After some rainfall from Hurricane Maria, PAHO reports that hospitals are considered to be structurally sound.
# The Dominican Republic National Emergency Operation Centre (NEOC), has placed eight provinces on red alert and another five provinces on yellow alert ahead of Hurricane Marias forecasted arrival as a major storm late on 20 September or early 21 September.


Create a chain that will extract the information and then parse it. This uses LangChain under the hood

In [ ]:
chain = create_extraction_chain(llm, schema)

In [ ]:
output = chain.invoke({"text": "Platte County is part of Wyoming."})
print(output["data"])

{'Triple': [{'Predicate': 'is part of', 'Entity 1': 'Platte County (GPE)', 'Entity 2': 'Wyoming (GPE)'}]}


In [ ]:
#input_file_path_training_data = '/content/drive/MyDrive/01-Research/Geo-Isa/Test/States_txt/NewHampshire-counties-preprocessed.txt'
input_file_path_training_data = "/content/drive/MyDrive/01-Research/Geo-Isa/Coding/evaluation-1600-annotated-tweets.txt"


# Read the content of the input file
# with open(input_file_path_training_data, 'r', encoding='utf-8') as file:
#     input_train_text = file.read()

with open(input_file_path_training_data, 'r', encoding='utf-8') as file:
    sentences = file.readlines() ## segmenting sentences

In [ ]:
import csv
import openai

# Define the path to the output folder on Google Drive
output_folder_path = '/content/drive/MyDrive/01-Research/Geo-Isa/Coding/'

# Open the CSV file in write mode inside the output folder
output_file_path = output_folder_path + 'GPT3.5_evaluation_1600_annotated_tweets.csv'
with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    # Create a CSV writer object
    csvwriter = csv.writer(csvfile)

    # Write the header row to the CSV file
    csvwriter.writerow(['Sentence', 'Predicate', 'Entity 1', 'Entity 2'])

    # Iterate through sentences in batches
    for sentence in sentences:
        # Use invoke instead of run
        output = chain.invoke({"text": sentence})

        # Check if the output is a dictionary with "Triple" key
        if output and isinstance(output, dict) and "Triple" in output:
            for triple_data in output["Triple"]:
                # Extract relevant information from the triple_data
                predicate = triple_data.get('Predicate', 'None')
                entity1 = triple_data.get('Entity 1', 'None')
                entity2 = triple_data.get('Entity 2', 'None')

                # Write the row to the CSV file
                csvwriter.writerow([sentence.strip(), predicate, entity1, entity2])
        else:
            # Output a default value if the format is incorrect
            print(f"Output for sentence '{sentence.strip()}' is not in the expected format.")
            csvwriter.writerow([sentence.strip(), 'None', 'None', 'None'])


Output for sentence 'Colorado Springs and most of El Paso County will start the week under a freeze warning Monday, according to the National Weather Service in Pueblo.' is not in the expected format.
Output for sentence 'My great-grandfather moved from Kentucky to Storm Lake, Buena Vista, Iowa in .' is not in the expected format.
Output for sentence 'With school soon to be back in session, we would like to take a moment to congratulate Trevor Beale of Caroline County who received a 1,000 scholarship from Virginia Sheriffs Institute (VSI).' is not in the expected format.
Output for sentence 'Ice storm warning continues in Maryland from Frederick County west until pm.' is not in the expected format.
Output for sentence 'Yesterday, announced that the first steel beam was installed at the new battery production plant for Honda and LG Energy Solution in Fayette County.' is not in the expected format.
Output for sentence 'Lyon County vs Dawson Springs - Kentucky High School Boys Basketball 

KeyboardInterrupt: 

In [ ]:
!pip install openai==0.28

In [ ]:
import csv
import os
import json
from typing import List, Dict, Any
from tqdm import tqdm
import openai

# Set OpenAI API key directly
openai.api_key = 'your key here'

# Define a natural language version of the schema to instruct the model
system_prompt = """
Your task is to recognize the triple (entity 1 - predicate - entity 2) connection in each sentence if they exist.
1. Identify the main action (Predicate) between two geographical entities.
2. Extract two entities (Entity 1 and Entity 2) and label them with GPE for nations, states, cities or LOC for non-GPE locations.
3. Return the triple as a JSON object.
Example:
Sentence: Middlesex County is located in New Jersey.
Output: {"Predicate": "is located in", "Entity 1": "Middlesex County (GPE)", "Entity 2": "New Jersey (GPE)"}
If no entities or actions are detected, return: {"Predicate": "None", "Entity 1": "None", "Entity 2": "None"}.
"""

# Define input and output file paths
input_file_path = "/content/drive/MyDrive/01-Research/Geo-Isa/Coding/evaluation-1600-annotated-tweets.txt"
output_folder_path = '/content/drive/MyDrive/01-Research/Geo-Isa/Coding/'
output_file_path = os.path.join(output_folder_path, 'extracted_triples.csv')

def extract_triples(sentence: str) -> List[Dict[str, str]]:
    try:
        # Use OpenAI GPT-3.5 to process the sentence
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": sentence}
            ]
        )

        # Parse the response content safely
        content = response.choices[0]['message']['content']

        # Try parsing content as JSON
        triples = json.loads(content)

        return triples if isinstance(triples, list) else [triples]

    except json.JSONDecodeError:
        print(f"Error parsing the response for sentence '{sentence}': {content}")
        return [{"Predicate": "None", "Entity 1": "None", "Entity 2": "None"}]

    except Exception as e:
        print(f"Error processing sentence '{sentence}': {str(e)}")
        return [{"Predicate": "None", "Entity 1": "None", "Entity 2": "None"}]

def main():
    try:
        # Read input sentences
        with open(input_file_path, 'r', encoding='utf-8') as file:
            sentences = file.readlines()

        # Process sentences and write to CSV file
        with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)

            # Write header
            writer.writerow(['Original Tweet', 'Relation', 'Entity 1', 'Entity 2'])

            # Initialize tqdm progress bar
            for sentence in tqdm(sentences, desc="Processing sentences", unit="sentence"):
                triples = extract_triples(sentence.strip())
                for triple in triples:
                    writer.writerow([
                        sentence.strip(),
                        triple.get('Predicate', 'None'),
                        triple.get('Entity 1', 'None'),
                        triple.get('Entity 2', 'None')
                    ])

        print(f"Processing complete. Output written to {output_file_path}")

    except IOError as e:
        print(f"Error reading input file or writing to output file: {str(e)}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Processing sentences:   8%|▊         | 140/1672 [02:06<22:09,  1.15sentence/s]

Error parsing the response for sentence '??': I'm here to help. Would you like to proceed with the task?


Processing sentences:  15%|█▍        | 245/1672 [03:38<21:36,  1.10sentence/s]

Error parsing the response for sentence 'MORE DETAILS: .': I'm sorry for the oversight. Could you please provide a sentence or more context for me to analyze and extract the entities and the main action (predicate) for you?


Processing sentences:  40%|███▉      | 667/1672 [09:52<15:15,  1.10sentence/s]

...

Error parsing the response for sentence '?': How can I assist you today?


Processing sentences:  99%|█████████▊| 1650/1672 [24:41<00:27,  1.25s/sentence]

Error parsing the response for sentence 'Learn more:': To learn more about extracting triples from text data using natural language processing techniques, you can explore resources on information extraction, named entity recognition, and relationship extraction. You can refer to literature on NLP libraries like NLTK (Natural Language Toolkit), spaCy, or Stanford NLP for practical implementations.

Additionally, you may want to look into research papers and tutorials on Open Information Extraction (OpenIE) systems, which aim to extract triples from unstructured text data without relying on predefined vocabularies or ontologies.

Practicing with NLP tasks such as entity recognition, relationship extraction, and knowledge graph construction can also help you gain a better understanding of how to extract structured information from text data.


Processing sentences: 100%|██████████| 1672/1672 [24:59<00:00,  1.11sentence/s]

Processing complete. Output written to /content/drive/MyDrive/01-Research/Geo-Isa/Coding/extracted_triples.csv


### STOP HERE: DEC 6

In [ ]:
for sentence in sentences:
    # Generate a conversation using gpt for each sentence
    output = chain.run(text=sentence)["data"]
    printOutput(output)

Streaming output truncated to the last 5000 lines.
   ]
}
{
   "Triple": [
      {
         "Entity 1": "None",
         "Entity 2": "None",
         "Predicate": "None"
      }
   ]
}
{
   "Triple": [
      {
         "Entity 1": "-",
         "Entity 2": "-",
         "Predicate": "-"
      },
      {
         "Entity 1": "Alabamas new prison",
         "Entity 2": "Elmore County",
         "Predicate": "under construction in"
      },
      {
         "Entity 1": "Alabama Corrections Institution Finance Authority (ACIFA)",
         "Entity 2": "new final guaranteed maximum price",
         "Predicate": "approved"
      }
   ]
}
{
   "Triple": [
      {
         "Entity 1": "Macon County",
         "Entity 2": "Alabama",
         "Predicate": "is sought for"
      }
   ]
}
{
   "Triple": [
      {
         "Entity 1": "None",
         "Entity 2": "None",
         "Predicate": "None"
      }
   ]
}
{
   "Triple": [
      {
         "Predicate: None": "Entity 1: Alabama prison finance 

In [ ]:
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters

punkt_param = PunktParameters()
abbreviation = ['u.s']
punkt_param.abbrev_types = set(abbreviation)
tokenizer = PunktSentenceTokenizer(punkt_param)

list_train_sent = tokenizer.tokenize(input_train_text)

In [ ]:
list_train_sent

['"The Key Underwood Coon Dog Memorial Graveyard, located in rural Colbert County, Alabama, was established in .',
 'It is explicitly reserved for the burial of coon dogs."',
 'The Alabama High School Athletic Association has issued a clarification of its Amateur Rule which recently led to the Tuscaloosa County being forced to forfeit a win earlier this season.',
 '"did NOT benefit enslaved black people case and point notice for the sale of enslaved black people from the estate of decedent William Jemison in Tuscaloosa County, Alabama."',
 'Live Alabama High School Football Thorsby VS Central of Coosa County Live Here : -Sep-23 zknz\nGame Day!',
 'Alabama High School Football Pike County VS Opp Live Now : -Sep-23 dtby\n"Handley vs Central of Clay County | Alabama High School Boys Football Varsity The Central of Clay County (Lineville, AL) varsity football team has a home non-conference game vs.',
 'Handley (Roanoke, AL) today p."',
 '"Handley vs Central of Clay County | Alabama High Sc

In [ ]:
# training dataset
output = chain.run(text=input_train_text)["data"]
printOutput(output)

{
   "Triple": [
      {
         "Entity 1: The Key Underwood Coon Dog Memorial Graveyard (LOC)": "Entity 1: The Alabama High School Athletic Association (ORG)",
         "Entity 2: Colbert County, Alabama (GPE)": "Entity 2: Tuscaloosa County (GPE)",
         "Predicate: located in": "Predicate: led to"
      },
      {
         "Entity 1: The Key Underwood Coon Dog Memorial Graveyard (LOC)": "Entity 1: None",
         "Entity 2: Colbert County, Alabama (GPE)": "Entity 2: None",
         "Predicate: located in": "Predicate: None"
      },
      {
         "Entity 1: The Key Underwood Coon Dog Memorial Graveyard (LOC)": "Entity 1: None",
         "Entity 2: Colbert County, Alabama (GPE)": "Entity 2: None",
         "Predicate: located in": "Predicate: None"
      },
      {
         "Entity 1: The Key Underwood Coon Dog Memorial Graveyard (LOC)": "Entity 1: None",
         "Entity 2: Colbert County, Alabama (GPE)": "Entity 2: None",
         "Predicate: located in": "Predicate: None"
 

In [ ]:
 [chain.run(text=x)["data"] for x in list_train_sent]

RateLimitError: ignored

In [ ]:
# Test dataset
input_test_path = '/content/drive/MyDrive/Colab Notebooks/GPT-4 NER and Relex/Dataset/input_sample_small.txt'

# Read the content of the input file
with open(input_test_path, 'r', encoding='utf-8') as file:
    input_test_text = file.read()

In [ ]:
punkt_param = PunktParameters()
abbreviation = ['u.s', 'd.c']
punkt_param.abbrev_types = set(abbreviation)
tokenizer = PunktSentenceTokenizer(punkt_param)

list_test_sent = tokenizer.tokenize(input_test_text)

In [ ]:
list_test_sent

['All local, State, and EMAC assets are exhausted or do not have the capacity to meet this requirement.',
 'The Department of Health and Human Services are not able to fulfill this requirement request.',
 'US Dept of Energy supports RRCC, IMAT, JFO for USVI Hurricane Irma energy sector response.',
 'Provide deployable temporary medical treatment facilities in support of disaster operations in response to Hurricane Irma in the US Territory of US Virgin Islands.',
 'DOD activates Federal Coordinating Centers (FCCs) and provides NDMS Patient Reception and Definitive Care in support of NDMS operations in the Caribbean and continental United States.',
 'The assessments are to be conducted at both the Rafael Hernandez Airport located in the Municipality of Aguadilla and the Jose Aponte de la Torre Airport located in the Municipality of Ceiba.',
 'The capability must also include movement and maintenance of commodities from the airfield apron to a logistic distribution point no greater than 1

In [ ]:
# GPT-4
[chain.run(text=x)["data"] for x in list_train_sent]

RateLimitError: ignored

In [ ]:
# GPT-4 turbo
 [chain.run(text=x)["data"] for x in list_test_sent]

[{'Triple extraction': [{'Action,Entity_1,Entity_2': 'No action detected,No entity 1 detected,No entity 2 detected'}]},
 {'Triple extraction': [{'Action': 'No action detected',
    'Entity_1': 'No entity 1 detected',
    'Entity_2': 'No entity 2 detected'}]},
 {'Triple extraction': [{'Action': 'supports',
    'Entity_1': 'US Dept of Energy (ORG)',
    'Entity_2': 'RRCC (ORG)'},
   {'Action': 'supports',
    'Entity_1': 'US Dept of Energy (ORG)',
    'Entity_2': 'IMAT (ORG)'},
   {'Action': 'supports',
    'Entity_1': 'US Dept of Energy (ORG)',
    'Entity_2': 'JFO (ORG)'}]},
 {'Triple extraction': [{'Action': 'provide',
    'Entity_1': 'No entity 1 detected',
    'Entity_2': 'US Virgin Islands (GPE)'}]},
 {'Triple extraction': [{'Action': 'activates',
    'Entity_1': 'DOD (ORG)',
    'Entity_2': 'Federal Coordinating Centers (FCCs) (ORG)'},
   {'Action': 'provides',
    'Entity_1': 'DOD (ORG)',
    'Entity_2': 'NDMS Patient Reception and Definitive Care (ORG)'},
   {'Action': 'supports